In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV ,KFold
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.calibration import LabelEncoder
from sklearn.metrics import accuracy_score



In [2]:
df = pd.read_csv('/Users/hneen./iCloud Drive (Archive)/Desktop/L9/Graduation project/colon_predicitions/Data/colon-dataset-processed.csv')
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df

,Class,Age,p16540_C/C,p16540_G/C,p16540_G/G,p16580_C/C,p16580_C/T,p16580_T/T,mdm2_G/G,mdm2_G/T,mdm2_T/T,GAL3_A/A,GAL3_C/A,GAL3_C/C,TIM1_C/C,TIM1_G/C,TIM1_G/G
0,1,49,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False
1,1,49,False,True,False,True,False,False,True,False,False,True,False,False,False,True,False
2,1,49,False,True,False,True,False,False,False,True,False,False,True,False,True,False,False
3,1,36,False,True,False,True,False,False,False,False,True,False,True,False,True,False,False
4,1,49,True,False,False,False,True,False,False,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0,81,False,True,False,True,False,False,False,False,True,False,False,True,False,False,True
111,0,73,True,False,False,True,False,False,False,False,True,False,False,True,False,False,True
112,0,56,True,False,False,True,False,False,False,True,False,False,False,True,False,True,False
113,0,74,False,True,False,False,True,False,True,False,False,False,False,True,False,False,True


In [3]:
X=df.drop('Class',axis=1)
y=df['Class']

In [4]:

#split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
random_state=123

In [5]:
#overampling
sm = SMOTE(random_state=random_state)
X_oversampled, y_oversampled = sm.fit_resample(X, y)

X_train_oversampled, X_test_oversampled, y_train_oversampled, y_test_oversampled = train_test_split(X_oversampled, y_oversampled, test_size=0.2, random_state=42)


In [6]:
#underampling
rus = RandomUnderSampler(random_state=random_state)
X_undersampled, y_undersampled = rus.fit_resample(X, y)
X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(X_undersampled, y_undersampled, test_size=0.2, random_state=42)

In [7]:
# def model_training(X_train, X_test, y_train, y_test):
#     # List of models to train

#     models = [
#         AdaBoostClassifier(),
#         ExtraTreesClassifier(random_state=random_state),
#         HistGradientBoostingClassifier(random_state=random_state),
#         LGBMClassifier(),
#         LogisticRegression(random_state=random_state),
#         RandomForestClassifier(max_depth=10, random_state=random_state,n_estimators=100),
#         SVC(random_state=random_state,kernel='linear',C=1),
#         XGBClassifier(random_state=random_state)
#     ]

#     # Create an empty DataFrame to store results
#     results_df = pd.DataFrame(columns=['Model', 'Accuracy'])

#     # Assuming you have your training data in X_train, y_train
#     # and your test data in X_test, y_test

#     for model in models:
#         # Train the model
#         model.fit(X_train, y_train)
        
#         # Make predictions (optional)
#         predictions = model.predict(X_test)
        
#         # Evaluate the model (optional)
#         accuracy = accuracy_score(y_test, predictions)
        
#         # Append results to the DataFrame
#         results_df = results_df.append({'Model': model.__class__.__name__, 'Accuracy': accuracy}, ignore_index=True)

#     return results_df.sort_values(by=['Accuracy'], ascending=False)


In [8]:
# from joblib import Parallel, delayed
# import pandas as pd
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier
# from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# from sklearn.ensemble import HistGradientBoostingClassifier
# from lightgbm import LGBMClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from xgboost import XGBClassifier

# def train_and_evaluate(model, X_train, X_test, y_train, y_test):
#     # Train the model
#     model.fit(X_train, y_train)
    
#     # Make predictions
#     predictions = model.predict(X_test)
    
#     # Evaluate the model
#     accuracy = accuracy_score(y_test, predictions)
    
#     return model.__class__.__name__, accuracy

# def model_training(X_train, X_test, y_train, y_test, random_state=42):
#     # List of models to train
#     models = [
#         AdaBoostClassifier(),
#         ExtraTreesClassifier(random_state=random_state, n_jobs=-1),
#         HistGradientBoostingClassifier(random_state=random_state),
#         LGBMClassifier(),
#         LogisticRegression(random_state=random_state, n_jobs=-1),
#         RandomForestClassifier(max_depth=10, random_state=random_state, n_estimators=100, n_jobs=-1),
#         SVC(random_state=random_state, kernel='linear', C=1),
#         XGBClassifier(random_state=random_state)
#     ]

#     # Parallelize the training and evaluation of models
#     results = Parallel(n_jobs=4)(delayed(train_and_evaluate)(model, X_train, X_test, y_train, y_test) for model in models)

#     # Convert results to a DataFrame
#     results_df = pd.DataFrame(results, columns=['Model', 'Accuracy'])

#     return results_df.sort_values(by='Accuracy', ascending=False)

# # Call the function with your data
# # results_df = model_training(X_train, X_test, y_train, y_test)


In [9]:
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

def model_training(X_train, X_test, y_train, y_test, random_state=42):
    # List of models to train
    models = [
        AdaBoostClassifier(),
        ExtraTreesClassifier(random_state=random_state, n_jobs=-1), # Parallel processing enabled
        HistGradientBoostingClassifier(random_state=random_state),
        LGBMClassifier(),
        LogisticRegression(random_state=random_state, n_jobs=-1), # Parallel processing enabled
        RandomForestClassifier(max_depth=10, random_state=random_state, n_estimators=100, n_jobs=-1), # Parallel processing enabled
        SVC(random_state=random_state, kernel='linear', C=1),
        XGBClassifier(random_state=random_state)
    ]

    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Model', 'Accuracy'])

    for model in models:
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        predictions = model.predict(X_test)
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, predictions)
        
        # Append results to the DataFrame
        results_df = results_df.append({'Model': model.__class__.__name__, 'Accuracy': accuracy}, ignore_index=True)

    # Sort and return the results DataFrame
    return results_df.sort_values(by=['Accuracy'], ascending=False)


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [10]:
model_training(X_train, X_test, y_train, y_test)

/var/folders/54/jmftjsf54wz5wws1xh5x5gww0000gn/T/ipykernel_1115/1773047890.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Model': model.__class__.__name__, 'Accuracy': accuracy}, ignore_index=True)
/var/folders/54/jmftjsf54wz5wws1xh5x5gww0000gn/T/ipykernel_1115/1773047890.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Model': model.__class__.__name__, 'Accuracy': accuracy}, ignore_index=True)
/var/folders/54/jmftjsf54wz5wws1xh5x5gww0000gn/T/ipykernel_1115/1773047890.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Model': model.__class__.__name__, 'Accuracy': accuracy}, ignore_index=True)


[LightGBM] [Info] Number of positive: 40, number of negative: 52
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000531 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38
[LightGBM] [Info] Number of data points in the train set: 92, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.434783 -> initscore=-0.262364
[LightGBM] [Info] Start training from score -0.262364
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/var/folders/54/jmftjsf54wz5wws1xh5x5gww0000gn/T/ipykernel_1115/1773047890.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Model': model.__class__.__name__, 'Accuracy': accuracy}, ignore_index=True)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/var/folders/54/jmftjsf54wz5wws1xh5x5gww0000gn/T/ipykernel_1115/1773047890.py:38: FutureWarning: The frame.append method is deprecated and will be rem

,Model,Accuracy
0,AdaBoostClassifier,0.869565
4,LogisticRegression,0.826087
6,SVC,0.826087
1,ExtraTreesClassifier,0.782609
2,HistGradientBoostingClassifier,0.782609
3,LGBMClassifier,0.782609
7,XGBClassifier,0.782609
5,RandomForestClassifier,0.739130


In [11]:
model_training(X_train_oversampled, X_test_oversampled, y_train_oversampled, y_test_oversampled)

/var/folders/54/jmftjsf54wz5wws1xh5x5gww0000gn/T/ipykernel_1115/1773047890.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Model': model.__class__.__name__, 'Accuracy': accuracy}, ignore_index=True)
/var/folders/54/jmftjsf54wz5wws1xh5x5gww0000gn/T/ipykernel_1115/1773047890.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Model': model.__class__.__name__, 'Accuracy': accuracy}, ignore_index=True)


In [ ]:
model_training(X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled)